# Gestor de investimento
---

In [2]:
import pandas as pd


# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [15]:
df_abr = pd.read_excel('../datasets/PosicaoHistorico_200091-20200430.xls')
df_jan = pd.read_excel('../datasets/PosicaoHistorico_200091-20200131.xls')
df_fev = pd.read_excel('../datasets/PosicaoHistorico_200091-20200228.xls')
df_mar = pd.read_excel('../datasets/PosicaoHistorico_200091-20200331.xls')

WARNING *** file size (71838) not 512 + multiple of sector size (512)
WARNING *** file size (70812) not 512 + multiple of sector size (512)
WARNING *** file size (69788) not 512 + multiple of sector size (512)
WARNING *** file size (71834) not 512 + multiple of sector size (512)


In [32]:
files = os.listdir('../datasets/')

for file_name in files:
    #print(file_name)
    df = pd.read_excel('../datasets/' + file_name)
    date_position = df[df['Unnamed: 56'].str.contains('Data de referência', na=False)]['Unnamed: 56']
    display(date_position)

WARNING *** file size (70812) not 512 + multiple of sector size (512)


3    Data de referência: 31/01/2020
Name: Unnamed: 56, dtype: object

WARNING *** file size (71834) not 512 + multiple of sector size (512)


3    Data de referência: 31/03/2020
Name: Unnamed: 56, dtype: object

WARNING *** file size (71838) not 512 + multiple of sector size (512)


3    Data de referência: 30/04/2020
Name: Unnamed: 56, dtype: object

WARNING *** file size (69788) not 512 + multiple of sector size (512)


3    Data de referência: 28/02/2020
Name: Unnamed: 56, dtype: object

In [16]:
df_abr[~df_abr['Unnamed: 2'].isnull()].head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84
16,nan,NaN,Ações,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,nan,74078,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
20,nan,NaN,Papel,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
24,nan,NaN,BPAC11,nan,nan,nan,nan,nan,nan,nan,200,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,NaN,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,NaN,200,nan,NaN,nan,NaN,nan,NaN,nan,42.30,NaN,nan,nan,nan,nan,NaN,"8,460.00",NaN
26,nan,NaN,JHSF3,nan,nan,nan,nan,nan,nan,nan,1600,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,NaN,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,NaN,1600,nan,NaN,nan,NaN,nan,NaN,nan,4.16,NaN,nan,nan,nan,nan,NaN,"6,656.00",NaN
28,nan,NaN,MGLU3,nan,nan,nan,nan,nan,nan,nan,800,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,NaN,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,NaN,800,nan,NaN,nan,NaN,nan,NaN,nan,49.70,NaN,nan,nan,nan,nan,NaN,"39,760.00",NaN
30,nan,NaN,PETR4,nan,nan,nan,nan,nan,nan,nan,400,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,NaN,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,NaN,400,nan,NaN,nan,NaN,nan,NaN,nan,18.05,NaN,nan,nan,nan,nan,NaN,"7,220.00",NaN
32,nan,NaN,WEGE3,nan,nan,nan,nan,nan,nan,nan,300,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,NaN,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,NaN,300,nan,NaN,nan,NaN,nan,NaN,nan,39.94,NaN,nan,nan,nan,nan,NaN,"11,982.00",NaN
35,nan,NaN,Opções,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,0,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
37,nan,NaN,Termos,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,n

In [19]:
# Função para pegar os dados de Papel (ações)
def get_stocks(df):
    stocks = []

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Papel' or row['Unnamed: 2'] == 'Ações':
            continue
        elif row['Unnamed: 2'] == 'Opções':
            break
        else:
            # continue
            #print(row[['Unnamed: 2', 'Unnamed: 10', 'Unnamed: 18']])
            stocks.append(row[['Unnamed: 2', 'Unnamed: 10', 'Unnamed: 18']].values)

    pd_stock = pd.DataFrame(columns=['Papel', 'Qtd. Disponivel', 'Qtde. Projetado'], data=stocks)

    return pd_stock

get_stocks(df_jan)

,Papel,Qtd. Disponivel,Qtde. Projetado
0,JHSF3,0,1600
1,MGLU3,800,0


In [23]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84
0,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
1,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,Histórico de Posição Consolidada,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
2,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,Mês: Abril/2020,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
3,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,Data de referência: 30/04/2020,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
4,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
5,nan,Gabriel Machado Lopes Goncalves,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,CPF/CNPJ,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,980.742.371-68,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
6,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
7,nan,"Rua Barata Ribeiro, 160 - Ap 908\nRio De Janei...",NaN,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,Conta XP,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,200091,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
8,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan